In [ ]:
!mkdir -p .data .output
!aws s3 cp s3://amazon-reviews-pds/tsv/sample_us.tsv .data/ 
!aws s3 cp s3://amazon-reviews-pds/tsv/sample_fr.tsv .data/ 

In [ ]:
from sagemaker import Session

session = Session()
bucket = session.default_bucket()
key_prefix = 'frameworkprocessors/huggingface-example'

source_path = session.upload_data('.data', bucket=bucket, key_prefix=f'{key_prefix}/data')
source_path

In [ ]:
from sagemaker.huggingface import HuggingFaceProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role

hfp = HuggingFaceProcessor(
    role=get_execution_role(), 
    instance_count=2,
    instance_type='ml.g4dn.xlarge',
    transformers_version='4.4.2',
    pytorch_version='1.6.0', 
    base_job_name='frameworkprocessor-hf'
)

In [ ]:
hfp.run(
    code='huggingface-processing.py',
    inputs=[
        ProcessingInput(
            input_name='data',
            source=source_path,
            destination='/opt/ml/processing/input/data/'
        )
    ],
    outputs=[
        ProcessingOutput(output_name='train', source='/opt/ml/processing/output/train/'),
        ProcessingOutput(output_name='test', source='/opt/ml/processing/output/test/'),
        ProcessingOutput(output_name='val', source='/opt/ml/processing/output/val/'),
    ],
    logs=False
)